In [1]:
import math
import os.path as path
import numpy as np
import pandas as pd
from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.callbacks import EarlyStopping
from keras.layers import Conv2D, BatchNormalization, Dropout, MaxPooling2D, Dense, Flatten, ZeroPadding2D, Activation
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras.layers import average, Input, Concatenate
from extra_functions import *


def load_and_format(in_path):
    out_df = pd.read_json(in_path)
    out_images = out_df.apply(lambda c_row: [np.stack([c_row['band_1'],c_row['band_2']], -1).reshape((75,75,2))],1)
    out_images = np.stack(out_images).squeeze()
    return out_df, out_images

dir_path = path.abspath(path.join('__file__',"../.."))
train_path = dir_path + "/train.json"
test_path = dir_path + "/test.json"

train_df, train_images = load_and_format(train_path)
test_df, test_images = load_and_format(test_path)
train_df.inc_angle = train_df.inc_angle.replace('na', 0)
train_df.inc_angle = train_df.inc_angle.astype(float).fillna(0.0)
x_angle_train = np.array(train_df.inc_angle)
print('angle shape')
print(x_angle_train.shape)
x_angle_test = np.array(test_df.inc_angle)

x_train, x_val, x_angle_train, x_angle_val, y_train, y_val = train_test_split(train_images
                    , x_angle_train, to_categorical(train_df['is_iceberg']), random_state=2017, train_size=0.75)

print('Train', x_train.shape, y_train.shape)
print('Validation', x_val.shape, y_val.shape)

"""
print("reading data")
x_train = pd.read_csv('x_train.csv', header=None)
x_train = x_train.values
y_train = pd.read_csv('y_train.csv', header=None)
y_train = y_train.values
x_val = pd.read_csv('x_val.csv', header=None)
x_val = x_val.values
y_val = pd.read_csv('y_val.csv', header=None)
y_val = y_val.values  

print("reshaping data")
x_train = np.reshape(x_train, (3366, 75, 75, 2))
x_val = np.reshape(x_val, (482, 75, 75, 2)) """

weight_decay = 0.05

image_input = Input(shape=(75, 75, 2), name="image")
angle_input = Input(shape=[1], name="angle")

cnn = BatchNormalization()(image_input)

cnn = Conv2D(16, kernel_size=(3,3), padding = 'same', activation='relu')(cnn)
cnn = Dropout(0.2, seed=25)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Conv2D(32, kernel_size=(3,3), padding = 'same', activation='relu')(cnn)
cnn = Dropout(0.3, seed=25)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', activation='relu')(cnn)
cnn = Dropout(0.3, seed=25)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Conv2D(64, kernel_size=(3,3), padding = 'same', activation='relu')(cnn)
cnn = Dropout(0.3, seed=25)(cnn)
cnn = MaxPooling2D(pool_size=(2,2))(cnn)

cnn = Flatten()(cnn)
concatenated_features = Concatenate()([cnn, BatchNormalization()(angle_input)])

cnn = Dense(250, activation='relu')(concatenated_features)
cnn = Dropout(0.4, seed=25)(cnn)

cnn = Dense(75, activation='relu')(cnn)
cnn = Dropout(0.4, seed=25)(cnn)

output = Dense(2, activation='softmax')(cnn)


model = Model(inputs=[image_input, angle_input], outputs=output)
model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()
print("Training")
early_stopping = EarlyStopping(monitor = 'val_loss', patience = 15)
model.fit([x_train, x_angle_train], y_train, batch_size = 64, validation_data = ([x_val, x_angle_val], y_val), 
          epochs = 250, shuffle = True, callbacks=[early_stopping])

print("predicting")
test_predictions = model.predict([test_images, x_angle_test])

pred_df = test_df[['id']].copy()
pred_df['is_iceberg'] = test_predictions[:,1]
print("creating csv")
pred_df.to_csv('predictions_2.csv', index = False)

Using TensorFlow backend.


angle shape
(1604,)
Train (1203, 75, 75, 2) (1203, 2)
Validation (401, 75, 75, 2) (401, 2)


/home/joseph/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
image (InputLayer)               (None, 75, 75, 2)     0                                            
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 75, 75, 2)     8           image[0][0]                      
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 75, 75, 16)    304         batch_normalization_1[0][0]      
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 75, 75, 16)    0           conv2d_1[0][0]                   
___________________________________________________________________________________________

1203/1203 [==============================] - 0s - loss: 0.1477 - acc: 0.9493 - val_loss: 0.2779 - val_acc: 0.8703
Epoch 25/250
1203/1203 [==============================] - 0s - loss: 0.1378 - acc: 0.9476 - val_loss: 0.2284 - val_acc: 0.9077
Epoch 26/250
1203/1203 [==============================] - 0s - loss: 0.1397 - acc: 0.9460 - val_loss: 0.2399 - val_acc: 0.8853
Epoch 27/250
1203/1203 [==============================] - 0s - loss: 0.1514 - acc: 0.9377 - val_loss: 0.2631 - val_acc: 0.8803
Epoch 28/250
1203/1203 [==============================] - 0s - loss: 0.1284 - acc: 0.9543 - val_loss: 0.2757 - val_acc: 0.8678
Epoch 29/250
1203/1203 [==============================] - 0s - loss: 0.1160 - acc: 0.9518 - val_loss: 0.2269 - val_acc: 0.8928
Epoch 30/250
1203/1203 [==============================] - 0s - loss: 0.1411 - acc: 0.9526 - val_loss: 0.2348 - val_acc: 0.8953
Epoch 31/250
1203/1203 [==============================] - 0s - loss: 0.1095 - acc: 0.9576 - val_loss: 0.2663 - val_acc: 0.86